## Titanic: Machine Learning from Disaster

In [129]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

SEED = 1234
np.random.seed = SEED

In [130]:
train, test = pd.read_csv(
    'data/titanic/train.csv'
    # путь к вашему файлу train
), pd.read_csv( 
    'data/titanic/test.csv'
    # путь к вашему файлу test
)

## Алгоритм 1 (Kaggle 0.72249)

In [131]:
# Извлечение Title
train['Title'] = train['Name'].apply(lambda x: x.split(', ')[1]).apply(lambda x: x.split('.')[0])
train['Title'] = train['Title'].replace(['Don','Dona', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col','Sir','Dr'],'Mr')
train['Title'] = train['Title'].replace(['Mlle','Ms'], 'Miss')
train['Title'] = train['Title'].replace(['the Countess','Mme','Lady','Dr'], 'Mrs')
df = pd.get_dummies(train['Title'],prefix='Title')
train = pd.concat([train,df],axis=1)

test['Title'] = test['Name'].apply(lambda x: x.split(', ')[1]).apply(lambda x: x.split('.')[0])
test['Title'] = test['Title'].replace(['Don','Dona', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col','Sir','Dr'],'Mr')
test['Title'] = test['Title'].replace(['Mlle','Ms'], 'Miss')
test['Title'] = test['Title'].replace(['the Countess','Mme','Lady','Dr'], 'Mrs')
df = pd.get_dummies(test['Title'],prefix='Title')
test = pd.concat([test,df],axis=1)

In [132]:
#Name_length
train['Name_Len'] = train['Name'].apply(lambda x: len(x))
train['Name_Len'] = pd.qcut(train['Name_Len'],5)

test['Name_Len'] = test['Name'].apply(lambda x: len(x))
test['Name_Len'] = pd.qcut(test['Name_Len'],5)

In [134]:
#Dead_female_family & Survive_male_family
train['Surname'] = train['Name'].apply(lambda x:x.split(',')[0])
#dead_female_surname = list(set(train[(train.Sex=='female') & (train.Age>=12)
#                              & (train.Survived==0) & ((train.Parch>0) | (train.SibSp > 0))]['Surname'].values))
#survive_male_surname = list(set(train[(train.Sex=='male') & (train.Age>=12)
#                              & (train.Survived==1) & ((train.Parch>0) | (train.SibSp > 0))]['Surname'].values))
#train['Dead_female_family'] = np.where(train['Surname'].isin(dead_female_surname),0,1)
#train['Survive_male_family'] = np.where(train['Surname'].isin(survive_male_surname),0,1)


train = train.drop(['Name','Surname'],axis=1)

test['Surname'] = test['Name'].apply(lambda x:x.split(',')[0])
#dead_female_surname = list(set(test[(test.Sex=='female') & (test.Age>=12)
#                               & ((test.Parch>0) | (test.SibSp > 0))]['Surname'].values))
#survive_male_surname = list(set(test[(test.Sex=='male') & (test.Age>=12)
#                               & ((test.Parch>0) | (test.SibSp > 0))]['Surname'].values))
#test['Dead_female_family'] = np.where(test['Surname'].isin(dead_female_surname),0,1)
#test['Survive_male_family'] = np.where(test['Surname'].isin(survive_male_surname),0,1)


test = test.drop(['Name','Surname'],axis=1)

In [135]:
#Age & isChild
group = train.groupby(['Title', 'Pclass'])['Age']
train['Age'] = group.transform(lambda x: x.fillna(x.median()))

train = train.drop('Title',axis=1)

train['IsChild'] = np.where(train['Age']<=12,1,0)
train['Age'] = pd.cut(train['Age'],5)
train = train.drop('Age',axis=1)

group = test.groupby(['Title', 'Pclass'])['Age']
test['Age'] = group.transform(lambda x: x.fillna(x.median()))

test = test.drop('Title',axis=1)

test['IsChild'] = np.where(test['Age']<=12,1,0)
test['Age'] = pd.cut(test['Age'],5)
test = test.drop('Age',axis=1)

In [136]:
#FamilySize
train['FamilySize'] = np.where(train['SibSp']+train['Parch']==0, 'Alone',
                                    np.where(train['SibSp']+train['Parch']<=3, 'Small', 'Big'))
df = pd.get_dummies(train['FamilySize'],prefix='FamilySize')
train = pd.concat([train,df],axis=1).drop(['SibSp','Parch','FamilySize'],axis=1)

test['FamilySize'] = np.where(test['SibSp']+test['Parch']==0, 'Alone',
                                    np.where(test['SibSp']+test['Parch']<=3, 'Small', 'Big'))
df = pd.get_dummies(test['FamilySize'],prefix='FamilySize')
test = pd.concat([test,df],axis=1).drop(['SibSp','Parch','FamilySize'],axis=1)

In [137]:
#Ticket
train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x)[0])
train['Ticket_Lett'] = train['Ticket_Lett'].apply(lambda x: str(x))

train['High_Survival_Ticket'] = np.where(train['Ticket_Lett'].isin(['1', '2', 'P']),1,0)
train['Low_Survival_Ticket'] = np.where(train['Ticket_Lett'].isin(['A','W','3','7']),1,0)
train = train.drop(['Ticket','Ticket_Lett'],axis=1)

test['Ticket_Lett'] = test['Ticket'].apply(lambda x: str(x)[0])
test['Ticket_Lett'] = test['Ticket_Lett'].apply(lambda x: str(x))

test['High_Survival_Ticket'] = np.where(test['Ticket_Lett'].isin(['1', '2', 'P']),1,0)
test['Low_Survival_Ticket'] = np.where(test['Ticket_Lett'].isin(['A','W','3','7']),1,0)
test = test.drop(['Ticket','Ticket_Lett'],axis=1)

In [138]:
#Embarked
train = train.drop('Embarked',axis=1)
test = test.drop('Embarked',axis=1)

In [139]:
#Cabin
train['Cabin_isNull'] = np.where(train['Cabin'].isnull(),0,1)
train = train.drop('Cabin',axis=1)

test['Cabin_isNull'] = np.where(test['Cabin'].isnull(),0,1)
test = test.drop('Cabin',axis=1)

In [140]:
#PClass
df = pd.get_dummies(train['Pclass'],prefix='Pclass')
train = pd.concat([train,df],axis=1).drop('Pclass',axis=1)

df = pd.get_dummies(test['Pclass'],prefix='Pclass')
test = pd.concat([test,df],axis=1).drop('Pclass',axis=1)

In [141]:
#Sex
df = pd.get_dummies(train['Sex'],prefix='Sex')
train = pd.concat([train,df],axis=1).drop('Sex',axis=1)

df = pd.get_dummies(test['Sex'],prefix='Sex')
test = pd.concat([test,df],axis=1).drop('Sex',axis=1)

In [142]:
#Fare
train['Fare'].fillna(train['Fare'].dropna().median(),inplace=True)
train['Low_Fare'] = np.where(train['Fare']<=8.662,1,0)
train['High_Fare'] = np.where(train['Fare']>=26,1,0)
train = train.drop('Fare',axis=1)

test['Fare'].fillna(test['Fare'].dropna().median(),inplace=True)
test['Low_Fare'] = np.where(test['Fare']<=8.662,1,0)
test['High_Fare'] = np.where(test['Fare']>=26,1,0)
test = test.drop('Fare',axis=1)

In [143]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

#Закодировали Name_Len цифрами вместо названий в test & train
encName_Len = preprocessing.LabelEncoder()
encName_Len.fit(pd.concat((test['Name_Len'], train['Name_Len'])))
test['Name_Len'] = encName_Len.transform(test['Name_Len'])
train['Name_Len'] = encName_Len.transform(train['Name_Len'])

In [144]:
train.head()

,PassengerId,Survived,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Name_Len,IsChild,FamilySize_Alone,FamilySize_Big,...,High_Survival_Ticket,Low_Survival_Ticket,Cabin_isNull,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Low_Fare,High_Fare
0,1,0,0,0,1,0,2,0,0,0,...,0,1,0,0,0,1,0,1,1,0
1,2,1,0,0,0,1,8,0,0,0,...,1,0,1,1,0,0,1,0,0,1
2,3,1,0,1,0,0,2,0,1,0,...,0,0,0,0,0,1,1,0,1,0
3,4,1,0,0,0,1,8,0,0,0,...,1,0,1,1,0,0,1,0,0,1
4,5,0,0,0,1,0,4,0,1,0,...,0,1,0,0,0,1,0,1,1,0


In [145]:
#Подготовили данные так, что X_tr - таблица без id и Survived, а в y_tr сохранены Survived
X_tr, y_tr = train.drop(['PassengerId', 'Survived'], axis=1), train['Survived']

In [146]:
#Сделаем скейлинг данных
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(X_tr)
#Преобразуем тренировочные данные
X_tr_scaled = scaler.transform(X_tr)
#Преобразуем тестовые данные
X_te = test.drop(['PassengerId'], axis=1)
X_te_scaled = scaler.transform(X_te)

In [152]:
from sklearn.preprocessing import PolynomialFeatures #Препроцессинг 
from lightgbm import LGBMClassifier

from sklearn.pipeline import make_pipeline

from sklearn.svm import SVC 
#SVC(probability=True)

pipe = make_pipeline(PolynomialFeatures(), LGBMClassifier())
pipe

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('lgbmclassifier', LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=-1,
        num_leaves=31, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=0, silent=True, subsample=1, subsample_for_bin=50000,
        subsample_freq=1))])

In [153]:
param_grid = {
    #'polynomialfeatures__degree': [1, 2, 3],
    #'lgbmclassifier__learning_rate': [0.1, 0.3, 0.4, 0.5, 0.7, 0.9],
    #'lgbmclassifier__max_bin': [100, 150, 200, 250],
    #'lgbmclassifier__n_estimators': [10, 15, 20, 25],
    #'lgbmclassifier__num_leaves': [10, 20, 30, 40, 50],
    'polynomialfeatures__degree': [3],
    #'svc__C': [1, 10, 100, 200, 400]  #10
    'lgbmclassifier__learning_rate': [0.9],
    'lgbmclassifier__max_bin': [100],
    'lgbmclassifier__n_estimators': [15],
    'lgbmclassifier__num_leaves': [30]
}

In [154]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)

clf.fit(X_tr_scaled, y_tr)
y_te = clf.predict(X_te_scaled)
print("Наилучшие параметры: {}".format(clf.best_params_))

Наилучшие параметры: {'lgbmclassifier__learning_rate': 0.9, 'lgbmclassifier__max_bin': 100, 'lgbmclassifier__n_estimators': 15, 'lgbmclassifier__num_leaves': 30, 'polynomialfeatures__degree': 3}


In [155]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=1234)

scores = cross_val_score(clf, X_tr_scaled, y_tr, cv=kf, n_jobs=-1)
print("Среднее значение кросс-валидации: {}".format(np.mean(scores)))

Среднее значение кросс-валидации: 0.8383654510074697


In [156]:
#for catboost
y_te = y_te.astype(np.int64)

ans = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_te})
ans.to_csv('ans_titanic.csv', index=False)

## Алгоритм 2 (Kaggle 0.77512)

In [333]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [334]:
#Удалим уж совсем бессмысленные столбцы
test = test.drop(['Name', 'Ticket', 'Cabin'], axis=1)
train = train.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [335]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [336]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [337]:
#train.Age.value_counts().sort_index()

In [338]:
train['SimpleAge'] = pd.cut(train.Age, bins=[0, 12, 23, 30, 45, 120], 
                   labels=['child', 'teen', 'young', 'adult', 'aged'])

test['SimpleAge'] = pd.cut(test.Age, bins=[0, 12, 23, 30, 45, 120], 
                   labels=['child', 'teen', 'young', 'adult', 'aged'])

test = test.drop(['Age'], axis=1)
train = train.drop(['Age'], axis=1)

In [339]:
test.head()

,PassengerId,Pclass,Sex,SibSp,Parch,Fare,Embarked,SimpleAge
0,892,3,male,0,0,7.8292,Q,adult
1,893,3,female,1,0,7.0000,S,aged
2,894,2,male,0,0,9.6875,Q,aged
3,895,3,male,0,0,8.6625,S,young
4,896,3,female,1,1,12.2875,S,teen


In [340]:
test.SimpleAge.value_counts().sort_index()

adult    80
aged     52
child    25
teen     89
young    86
Name: SimpleAge, dtype: int64

In [341]:
#train.Fare.value_counts().sort_index()

In [342]:
train['SimpleFare'] = pd.cut(train.Fare, bins=[0, 8, 16, 32, 64, 128, 256, 514], 
                   labels=['one', 'two', 'three', 'four', 'five', 'six', 'seven'])

test['SimpleFare'] = pd.cut(test.Fare, bins=[0, 8, 16, 32, 64, 128, 256, 514], 
                   labels=['one', 'two', 'three', 'four', 'five', 'six', 'seven'])

test = test.drop(['Fare'], axis=1)
train = train.drop(['Fare'], axis=1)

In [343]:
train.head()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Embarked,SimpleAge,SimpleFare
0,1,0,3,male,1,0,S,teen,one
1,2,1,1,female,1,0,C,adult,five
2,3,1,3,female,0,0,S,young,one
3,4,1,1,female,1,0,S,adult,four
4,5,0,3,male,0,0,S,adult,two


In [344]:
train.SimpleFare.value_counts().sort_index()

five      80
four      93
one      226
seven      9
six       29
three    195
two      244
Name: SimpleFare, dtype: int64

In [345]:
train.dtypes

PassengerId       int64
Survived          int64
Pclass            int64
Sex              object
SibSp             int64
Parch             int64
Embarked         object
SimpleAge      category
SimpleFare     category
dtype: object

In [346]:
#Закодировали Sex цифрами вместо названий в test & train
encSex = preprocessing.LabelEncoder()
encSex.fit(pd.concat((test['Sex'], train['Sex'])))
test['Sex'] = encSex.transform(test['Sex'])
train['Sex'] = encSex.transform(train['Sex'])

In [347]:
train.head()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Embarked,SimpleAge,SimpleFare
0,1,0,3,1,1,0,S,teen,one
1,2,1,1,0,1,0,C,adult,five
2,3,1,3,0,0,0,S,young,one
3,4,1,1,0,1,0,S,adult,four
4,5,0,3,1,0,0,S,adult,two


In [348]:
train.loc[train.Embarked.isnull(), 'Embarked'] = 'Unknown'
test.loc[test.Embarked.isnull(), 'Embarked'] = 'Unknown'

In [349]:
test.Embarked.value_counts().sort_index()

C    102
Q     46
S    270
Name: Embarked, dtype: int64

In [350]:
#Закодировали Embarked цифрами вместо названий в test & train
encEmbarked = preprocessing.LabelEncoder()
encEmbarked.fit(pd.concat((test['Embarked'], train['Embarked'])))
test['Embarked'] = encEmbarked.transform(test['Embarked'])
train['Embarked'] = encEmbarked.transform(train['Embarked'])

In [351]:
test.head()

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,SimpleAge,SimpleFare
0,892,3,1,0,0,1,adult,one
1,893,3,0,1,0,2,aged,one
2,894,2,1,0,0,1,aged,two
3,895,3,1,0,0,2,young,two
4,896,3,0,1,1,2,teen,two


In [352]:
# Перевод категорий в числа

test.SimpleAge = test.SimpleAge.astype('category').cat.codes
test.SimpleFare = test.SimpleFare.astype('category').cat.codes

train.SimpleAge = train.SimpleAge.astype('category').cat.codes
train.SimpleFare = train.SimpleFare.astype('category').cat.codes

In [353]:
test.dtypes

PassengerId    int64
Pclass         int64
Sex            int64
SibSp          int64
Parch          int64
Embarked       int64
SimpleAge       int8
SimpleFare      int8
dtype: object

In [354]:
#Убираем тип category с помощью get_dummies, т.е. one hot encode
#cols_to_transform = [ 'SimpleAge', 'SimpleFare' ]
#train = pd.get_dummies(train, columns = cols_to_transform )
#test = pd.get_dummies(test, columns = cols_to_transform )

In [355]:
train.head()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Embarked,SimpleAge,SimpleFare
0,1,0,3,1,1,0,2,3,2
1,2,1,1,0,1,0,0,0,0
2,3,1,3,0,0,0,2,4,2
3,4,1,1,0,1,0,2,0,1
4,5,0,3,1,0,0,2,0,6


In [356]:
#Подготовили данные так, что X_tr - таблица без id и Survived, а в y_tr сохранены Survived
X_tr, y_tr = train.drop(['PassengerId', 'Survived'], axis=1), train['Survived']

In [376]:
#Сделаем скейлинг данных
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(X_tr)
#Преобразуем тренировочные данные
X_tr_scaled = scaler.transform(X_tr)
#Преобразуем тестовые данные
X_te = test.drop(['PassengerId'], axis=1)
X_te_scaled = scaler.transform(X_te)

In [382]:
from sklearn.preprocessing import PolynomialFeatures #Препроцессинг 
from lightgbm import LGBMClassifier

from sklearn.pipeline import make_pipeline

from sklearn.svm import SVC 
#SVC(probability=True)

pipe = make_pipeline(PolynomialFeatures(), LGBMClassifier())
pipe

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('lgbmclassifier', LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=-1,
        num_leaves=31, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=0, silent=True, subsample=1, subsample_for_bin=50000,
        subsample_freq=1))])

In [383]:
param_grid = {
    #'polynomialfeatures__degree': [1, 2, 3],
    #'lgbmclassifier__learning_rate': [0.1, 0.3, 0.5, 0.7, 0.9],
    #'lgbmclassifier__max_bin': [200, 250, 300, 400, 500]
    #'lgbmclassifier__n_estimators': [110, 120, 130, 140, 150, 160],
    #'lgbmclassifier__num_leaves': [110, 120, 130, 140]
    'polynomialfeatures__degree': [2],
    #'svc__C': [1, 10, 100, 200, 400]  #10
    'lgbmclassifier__learning_rate': [0.4],
    'lgbmclassifier__max_bin': [150],
    'lgbmclassifier__n_estimators': [10],
    'lgbmclassifier__num_leaves': [45]
}

In [384]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)

clf.fit(X_tr_scaled, y_tr)
y_te = clf.predict(X_te_scaled)
print("Наилучшие параметры: {}".format(clf.best_params_))

Наилучшие параметры: {'lgbmclassifier__learning_rate': 0.4, 'lgbmclassifier__max_bin': 150, 'lgbmclassifier__n_estimators': 10, 'lgbmclassifier__num_leaves': 45, 'polynomialfeatures__degree': 2}


In [385]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=1234)

scores = cross_val_score(clf, X_tr_scaled, y_tr, cv=kf, n_jobs=-1)
print("Среднее значение кросс-валидации: {}".format(np.mean(scores)))

Среднее значение кросс-валидации: 0.8181595631159375


In [386]:
#for catboost
y_te = y_te.astype(np.int64)

ans = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_te})
ans.to_csv('ans_titanic.csv', index=False)